In [5]:
import json

response = {
  "type": "message",
  "role": "assistant",
  "content": [
    {
      "type": "text",
      "text": "# 📚 Model Monitoring with Scikit-Learn: Theory & Practice Guide\n\n## สารบัญ (Table of Contents)\n1. [บทนำ: Model Monitoring คืออะไร?](#1-บทนำ-model-monitoring-คืออะไร)\n2. [Section 1: Data Quality Monitoring](#section-1-data-quality-monitoring)\n3. [Section 2: Model Performance Tracking](#section-2-model-performance-tracking)\n4. [Section 3: Target Drift Detection](#section-3-target-drift-detection)\n5. [Section 4: Building Monitoring Dashboard](#section-4-building-monitoring-dashboard)\n6. [Best Practices และแนวทางปฏิบัติ](#best-practices-และแนวทางปฏิบัติ)\n\n---\n\n## 1. บทนำ: Model Monitoring คืออะไร?\n\n### 1.1 ความหมายและความสำคัญ\n\n**Model Monitoring** คือกระบวนการติดตามและเฝ้าระวังประสิทธิภาพของโมเดล Machine Learning หลังจากที่ได้ Deploy ไปใช้งานจริง (Production) เพื่อให้มั่นใจว่าโมเดลยังคงทำงานได้อย่างถูกต้องและน่าเชื่อถือ\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                    ML Model Lifecycle                           │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                 │\n│  Training → Validation → Deployment → [MONITORING] → Retrain   │\n│                                            ↑                    │\n│                                            │                    │\n│                              ┌─────────────┴─────────────┐      │\n│                              │   • Data Quality          │      │\n│                              │   • Performance Metrics   │      │\n│                              │   • Drift Detection       │      │\n│                              │   • Alerts & Actions      │      │\n│                              └───────────────────────────┘      │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n### 1.2 ทำไมต้อง Monitor โมเดล?\n\n| ปัญหา | คำอธิบาย | ผลกระทบ |\n|-------|----------|---------|\n| **Data Drift** | ข้อมูลในโลกจริงเปลี่ยนแปลงตลอดเวลา | โมเดลทำนายผิดพลาดมากขึ้น |\n| **Concept Drift** | ความสัมพันธ์ระหว่าง features และ target เปลี่ยนไป | โมเดลไม่สามารถเรียนรู้ pattern ใหม่ |\n| **Model Degradation** | ประสิทธิภาพโมเดลลดลงเมื่อเวลาผ่านไป | Business Impact สูง |\n| **Data Quality Issues** | ปัญหา missing values, outliers, duplicates | Garbage In, Garbage Out |\n\n### 1.3 องค์ประกอบหลักของ Model Monitoring\n\n```python\n# โครงสร้างหลักของ Monitoring System\nmonitoring_components = {\n    \"Data Quality\": [\"Missing Values\", \"Duplicates\", \"Outliers\", \"Schema Validation\"],\n    \"Performance\": [\"Accuracy\", \"Precision\", \"Recall\", \"F1\", \"AUC-ROC\"],\n    \"Drift Detection\": [\"Feature Drift\", \"Target Drift\", \"Prediction Drift\"],\n    \"Alerting\": [\"Threshold Alerts\", \"Anomaly Alerts\", \"Trend Alerts\"]\n}\n```\n\n---\n\n## Section 1: Data Quality Monitoring\n\n### 1.1 ทฤษฎี: Data Quality คืออะไร?\n\n**Data Quality** หมายถึงระดับความถูกต้อง ครบถ้วน และเชื่อถือได้ของข้อมูล หลักการสำคัญคือ **\"Garbage In, Garbage Out\"** - ถ้าข้อมูลไม่ดี โมเดลก็ไม่สามารถทำนายได้ดี\n\n#### มิติของ Data Quality (Data Quality Dimensions)\n\n```\n┌────────────────────────────────────────────────────────────┐\n│                   DATA QUALITY DIMENSIONS                  │\n├────────────────────────────────────────────────────────────┤\n│                                                            │\n│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │\n│  │ Completeness│  │  Accuracy   │  │ Consistency │        │\n│  │ (ครบถ้วน)   │  │ (ถูกต้อง)   │  │ (สอดคล้อง)  │        │\n│  └─────────────┘  └─────────────┘  └─────────────┘        │\n│                                                            │\n│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │\n│  │  Timeliness │  │   Validity  │  │  Uniqueness │        │\n│  │ (ทันเวลา)   │  │ (ถูกรูปแบบ) │  │ (ไม่ซ้ำซ้อน)│        │\n│  └─────────────┘  └─────────────┘  └─────────────┘        │\n└────────────────────────────────────────────────────────────┘\n```\n\n### 1.2 การตรวจสอบ Missing Values\n\n**Missing Values** คือค่าที่หายไปหรือไม่มีการบันทึกในข้อมูล สามารถแบ่งได้ 3 ประเภท:\n\n| ประเภท | คำอธิบาย | ตัวอย่าง |\n|--------|----------|----------|\n| **MCAR** (Missing Completely at Random) | ข้อมูลหายไปแบบสุ่มทั้งหมด | เซ็นเซอร์เสียแบบสุ่ม |\n| **MAR** (Missing at Random) | ข้อมูลหายขึ้นกับตัวแปรอื่น | รายได้หายเฉพาะกลุ่มอายุน้อย |\n| **MNAR** (Missing Not at Random) | ข้อมูลหายขึ้นกับค่าตัวเอง | คนรายได้สูงไม่กรอกรายได้ |\n\n#### Code Implementation:\n\n```python\nclass DataQualityMonitor:\n    def check_missing_values(self):\n        \"\"\"ตรวจสอบ missing values\"\"\"\n        missing = self.data.isnull().sum()\n        missing_pct = (missing / len(self.data) * 100).round(2)\n        \n        missing_df = pd.DataFrame({\n            'column': missing.index,\n            'missing_count': missing.values,\n            'missing_percentage': missing_pct.values\n        })\n        # กรองเฉพาะ columns ที่มี missing\n        missing_df = missing_df[missing_df['missing_count'] > 0].sort_values(\n            'missing_percentage', ascending=False\n        )\n        return missing_df\n```\n\n**หลักการ:**\n- ใช้ `isnull().sum()` นับจำนวน missing ในแต่ละ column\n- คำนวณ % โดยหารด้วยจำนวน rows ทั้งหมด\n- เรียงลำดับจากมากไปน้อยเพื่อให้เห็นปัญหาหลักก่อน\n\n### 1.3 การตรวจสอบ Duplicates\n\n**Duplicates** คือข้อมูลที่ซ้ำกัน อาจเกิดจาก:\n- Data entry ซ้ำ\n- ETL process มีปัญหา\n- System integration ผิดพลาด\n\n```python\ndef check_duplicates(self):\n    \"\"\"ตรวจสอบข้อมูลซ้ำ\"\"\"\n    # ตรวจสอบ duplicate rows ทั้งหมด\n    duplicate_rows = self.data.duplicated().sum()\n    \n    # ตรวจสอบ duplicate ตาม ID (ถ้ามี)\n    id_columns = [col for col in self.data.columns if 'id' in col.lower()]\n    duplicate_ids = {}\n    \n    for id_col in id_columns:\n        dup_count = self.data[id_col].duplicated().sum()\n        if dup_count > 0:\n            duplicate_ids[id_col] = int(dup_count)\n    \n    return {\n        'duplicate_rows': int(duplicate_rows),\n        'duplicate_percentage': round(duplicate_rows / len(self.data) * 100, 2),\n        'duplicate_ids': duplicate_ids\n    }\n```\n\n### 1.4 การตรวจหา Outliers\n\n**Outliers** คือค่าที่ผิดปกติหรืออยู่ห่างจากค่าส่วนใหญ่ มี 2 วิธีหลักในการตรวจจับ:\n\n#### วิธีที่ 1: IQR Method (Interquartile Range)\n\n```\n                    IQR Method\n    ◄───────────────────────────────────────────►\n    \n    Lower Bound                      Upper Bound\n         │                                │\n         ▼                                ▼\n    Q1 - 1.5×IQR                   Q3 + 1.5×IQR\n         │                                │\n    ─────┴────────┬────────┬────────┬─────┴─────\n                  Q1       Q2       Q3\n                  │        │        │\n                  ◄────────┴────────►\n                        IQR\n```\n\n**สูตร:**\n- IQR = Q3 - Q1\n- Lower Bound = Q1 - 1.5 × IQR\n- Upper Bound = Q3 + 1.5 × IQR\n\n#### วิธีที่ 2: Z-Score Method\n\n```python\n# Z-score = (x - μ) / σ\n# ถ้า |Z-score| > 3 → Outlier\n```\n\n#### Code Implementation:\n\n```python\ndef detect_outliers(self, method='iqr', threshold=1.5):\n    \"\"\"ตรวจหา outliers\"\"\"\n    numeric_cols = self.data.select_dtypes(include=[np.number]).columns\n    outlier_info = []\n    \n    for col in numeric_cols:\n        if method == 'iqr':\n            Q1 = self.data[col].quantile(0.25)\n            Q3 = self.data[col].quantile(0.75)\n            IQR = Q3 - Q1\n            lower_bound = Q1 - threshold * IQR\n            upper_bound = Q3 + threshold * IQR\n            outliers = self.data[(self.data[col] < lower_bound) | \n                                 (self.data[col] > upper_bound)]\n        else:  # z-score\n            z_scores = np.abs(stats.zscore(self.data[col].dropna()))\n            outliers = self.data[z_scores > threshold]\n        \n        outlier_info.append({\n            'column': col,\n            'outlier_count': len(outliers),\n            'outlier_percentage': round(len(outliers) / len(self.data) * 100, 2)\n        })\n    \n    return pd.DataFrame(outlier_info)\n```\n\n### 1.5 Data Quality Alert System\n\nระบบแจ้งเตือนใช้ **Threshold-based Approach** โดยกำหนดค่า threshold สำหรับแต่ละ metric:\n\n```python\nclass DataQualityAlert:\n    def __init__(self):\n        self.thresholds = {\n            'missing_rate': 0.05,      # ไม่เกิน 5% missing\n            'duplicate_rate': 0.01,    # ไม่เกิน 1% duplicates\n            'outlier_rate': 0.05       # ไม่เกิน 5% outliers\n        }\n```\n\n```\n┌─────────────────────────────────────────────────────────┐\n│                 ALERT SEVERITY LEVELS                   │\n├─────────────────────────────────────────────────────────┤\n│                                                         │\n│  🟢 HEALTHY    │  All metrics within threshold         │\n│                                                         │\n│  🟡 WARNING    │  Some metrics slightly above threshold│\n│                │  (1x - 2x threshold)                  │\n│                                                         │\n│  🔴 CRITICAL   │  Metrics significantly above threshold│\n│                │  (> 2x threshold)                     │\n└─────────────────────────────────────────────────────────┘\n```\n\n---\n\n## Section 2: Model Performance Tracking\n\n### 2.1 ทฤษฎี: Classification Metrics\n\nสำหรับ **Classification Problems** เราใช้ metrics ดังนี้:\n\n#### Confusion Matrix\n\n```\n                    PREDICTED\n                 │  Negative  │  Positive\n         ────────┼────────────┼───────────\n         Negative│     TN     │    FP     \n  ACTUAL ────────┼────────────┼───────────\n         Positive│     FN     │    TP     \n```\n\n| Metric | สูตร | ความหมาย |\n|--------|------|----------|\n| **Accuracy** | (TP + TN) / Total | ความถูกต้องโดยรวม |\n| **Precision** | TP / (TP + FP) | จากที่ทำนาย Positive มีกี่ % ที่ถูก |\n| **Recall** | TP / (TP + FN) | จาก Positive จริง ทำนายถูกกี่ % |\n| **F1-Score** | 2 × (P × R) / (P + R) | Harmonic Mean ของ Precision และ Recall |\n| **Specificity** | TN / (TN + FP) | จาก Negative จริง ทำนายถูกกี่ % |\n\n#### เมื่อไหร่ใช้ Metric ไหน?\n\n```\n┌────────────────────────────────────────────────────────────────┐\n│                    METRIC SELECTION GUIDE                      │\n├────────────────────────────────────────────────────────────────┤\n│                                                                │\n│  Use Case                    │  Primary Metric                 │\n│  ────────────────────────────┼────────────────────────────────│\n│  Balanced Dataset            │  Accuracy, F1-Score            │\n│  Imbalanced Dataset          │  F1-Score, AUC-ROC             │\n│  Cost of FP is high          │  Precision                     │\n│  Cost of FN is high          │  Recall                        │\n│  Fraud Detection             │  Recall, F1-Score              │\n│  Medical Diagnosis           │  Recall (Sensitivity)          │\n│  Spam Detection              │  Precision                     │\n└────────────────────────────────────────────────────────────────┘\n```\n\n### 2.2 Code Implementation: Performance Monitor\n\n```python\nclass ModelPerformanceMonitor:\n    def calculate_classification_metrics(self, y_true, y_pred, y_prob=None):\n        \"\"\"คำนวณ metrics สำหรับ classification\"\"\"\n        metrics = {\n            'accuracy': accuracy_score(y_true, y_pred),\n            'precision': precision_score(y_true, y_pred, average='weighted'),\n            'recall': recall_score(y_true, y_pred, average='weighted'),\n            'f1': f1_score(y_true, y_pred, average='weighted'),\n        }\n        \n        # Confusion matrix values\n        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()\n        metrics['true_positive'] = int(tp)\n        metrics['true_negative'] = int(tn)\n        metrics['false_positive'] = int(fp)\n        metrics['false_negative'] = int(fn)\n        \n        # Specificity\n        metrics['specificity'] = tn / (tn + fp) if (tn + fp) > 0 else 0\n        \n        return metrics\n```\n\n### 2.3 ทฤษฎี: Regression Metrics\n\nสำหรับ **Regression Problems**:\n\n| Metric | สูตร | คุณสมบัติ |\n|--------|------|----------|\n| **MAE** | Σ\\|yᵢ - ŷᵢ\\| / n | ค่าเฉลี่ยความคลาดเคลื่อนสัมบูรณ์, robust ต่อ outliers |\n| **MSE** | Σ(yᵢ - ŷᵢ)² / n | ลงโทษ error ใหญ่มากกว่า |\n| **RMSE** | √MSE | อยู่ในหน่วยเดียวกับ target |\n| **R²** | 1 - (SS_res / SS_tot) | สัดส่วนความแปรปรวนที่อธิบายได้ (0-1) |\n| **MAPE** | Σ\\|(yᵢ - ŷᵢ)/yᵢ\\| × 100 / n | % error, ระวัง division by zero |\n\n```python\ndef calculate_regression_metrics(self, y_true, y_pred):\n    \"\"\"คำนวณ metrics สำหรับ regression\"\"\"\n    return {\n        'mae': mean_absolute_error(y_true, y_pred),\n        'mse': mean_squared_error(y_true, y_pred),\n        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),\n        'r2': r2_score(y_true, y_pred),\n        'mape': np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100\n    }\n```\n\n### 2.4 Baseline Comparison\n\nการเปรียบเทียบกับ **Baseline** คือหัวใจของ Performance Monitoring:\n\n```\n┌─────────────────────────────────────────────────────────────┐\n│              PERFORMANCE COMPARISON FRAMEWORK               │\n├─────────────────────────────────────────────────────────────┤\n│                                                             │\n│   BASELINE                      CURRENT                     │\n│   (Training/Validation)         (Production)                │\n│   ┌───────────────┐             ┌───────────────┐           │\n│   │ Accuracy: 0.92│ ─────────►  │ Accuracy: 0.87│           │\n│   │ F1: 0.89      │  Compare    │ F1: 0.82      │           │\n│   └───────────────┘             └───────────────┘           │\n│                                                             │\n│   Degradation = (0.92 - 0.87) / 0.92 × 100 = 5.4%          │\n│                                                             │\n│   Status: 🟡 WARNING (degradation > 5%)                     │\n└─────────────────────────────────────────────────────────────┘\n```\n\n```python\ndef _compare_with_baseline(self, current_metrics, task):\n    \"\"\"เปรียบเทียบกับ baseline\"\"\"\n    comparison = {}\n    \n    for metric in key_metrics:\n        baseline_val = self.baseline_metrics.get(metric, 0)\n        current_val = current_metrics.get(metric, 0)\n        \n        change = current_val - baseline_val\n        change_pct = (change / baseline_val * 100) if baseline_val != 0 else 0\n        \n        comparison[metric] = {\n            'baseline': baseline_val,\n            'current': current_val,\n            'change': change,\n            'change_pct': change_pct,\n            'status': 'improved' if change > 0 else 'degraded' if change < 0 else 'stable'\n        }\n    \n    return comparison\n```\n\n### 2.5 Performance Degradation Alerts\n\n```\n┌─────────────────────────────────────────────────────────────┐\n│             PERFORMANCE DEGRADATION THRESHOLDS              │\n├─────────────────────────────────────────────────────────────┤\n│                                                             │\n│   Degradation %      │  Status    │  Action                 │\n│   ───────────────────┼────────────┼────────────────────────│\n│   < 5%               │  🟢 OK     │  Continue monitoring    │\n│   5% - 10%           │  🟡 WARN   │  Investigate cause      │\n│   > 10%              │  🔴 CRIT   │  Consider retraining    │\n│   > 20%              │  ⚫ FATAL  │  Immediate retrain      │\n│                                                             │\n└─────────────────────────────────────────────────────────────┘\n```\n\n---\n\n## Section 3: Target Drift Detection\n\n### 3.1 ทฤษฎี: Types of Drift\n\n**Drift** คือการเปลี่ยนแปลงของ data distribution ตามเวลา แบ่งได้ 3 ประเภทหลัก:\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                      TYPES OF DRIFT                             │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                 │\n│  1. DATA DRIFT (Covariate Drift)                               │\n│     P(X) changes, but P(Y|X) remains the same                  │\n│     ตัวอย่าง: ลูกค้ากลุ่มใหม่เข้ามา แต่ pattern ยังเหมือนเดิม   │\n│                                                                 │\n│  2. CONCEPT DRIFT                                               │\n│     P(Y|X) changes                                              │\n│     ตัวอย่าง: พฤติกรรมการผิดนัดชำระเปลี่ยนไป                   │\n│                                                                 │\n│  3. TARGET DRIFT (Label Drift)                                  │\n│     P(Y) changes                                                │\n│     ตัวอย่าง: อัตราการ default เพิ่มขึ้นในช่วง recession       │\n│                                                                 │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n#### Drift Patterns Over Time\n\n```\n1. SUDDEN DRIFT                2. GRADUAL DRIFT\n   │                              │\n   │    ┌────────                 │         ╱────\n   │    │                         │       ╱\n   │────┘                         │     ╱\n   │                              │   ╱\n   └────────────►                 └──────────────►\n        time                           time\n\n3. INCREMENTAL DRIFT           4. RECURRING DRIFT\n   │                              │\n   │          ╱──                 │  ╱╲    ╱╲\n   │        ╱                     │ ╱  ╲  ╱  ╲\n   │      ╱                       │╱    ╲╱    ╲\n   │────╱                         │\n   └────────────►                 └──────────────►\n        time                           time\n```\n\n### 3.2 Statistical Tests สำหรับ Drift Detection\n\n#### 3.2.1 Kolmogorov-Smirnov Test (KS Test)\n\nใช้สำหรับ **continuous variables**\n\n**หลักการ:**\n- เปรียบเทียบ Cumulative Distribution Function (CDF) ของสอง samples\n- H₀: ทั้งสอง distributions มาจากประชากรเดียวกัน\n- ถ้า p-value < 0.05 → Reject H₀ → มี drift\n\n```python\ndef ks_test(self, reference_col, current_col):\n    \"\"\"Kolmogorov-Smirnov Test\"\"\"\n    statistic, p_value = ks_2samp(reference_col.dropna(), current_col.dropna())\n    return {\n        'test': 'Kolmogorov-Smirnov',\n        'statistic': statistic,  # Max difference between CDFs\n        'p_value': p_value,\n        'drift_detected': p_value < 0.05\n    }\n```\n\n```\nKS Test Visualization:\n\n     CDF\n     1.0 ┤          ●●●●●●●●●●●●\n         │       ●●●       ○○○○○○\n         │    ●●●      ○○○○\n     0.5 ┤  ●●●    ○○○○  ← D = max difference\n         │ ●●   ○○○\n         │●  ○○○\n     0.0 ┼○○○─────────────────────►\n                Value\n         \n         ● Reference  ○ Current\n```\n\n#### 3.2.2 Chi-Square Test\n\nใช้สำหรับ **categorical variables**\n\n```python\ndef chi_square_test(self, reference_col, current_col):\n    \"\"\"Chi-Square Test สำหรับ categorical variables\"\"\"\n    # สร้าง contingency table\n    ref_counts = reference_col.value_counts()\n    curr_counts = current_col.value_counts()\n    \n    all_categories = set(ref_counts.index) | set(curr_counts.index)\n    \n    ref_freq = [ref_counts.get(cat, 0) for cat in all_categories]\n    curr_freq = [curr_counts.get(cat, 0) for cat in all_categories]\n    \n    contingency = np.array([ref_freq, curr_freq])\n    \n    chi2, p_value, dof, expected = chi2_contingency(contingency)\n    \n    return {\n        'test': 'Chi-Square',\n        'statistic': chi2,\n        'p_value': p_value,\n        'drift_detected': p_value < 0.05\n    }\n```\n\n#### 3.2.3 Population Stability Index (PSI)\n\n**PSI** เป็น metric ที่นิยมใช้ในอุตสาหกรรมการเงิน\n\n**สูตร:**\n```\nPSI = Σ (Actual% - Expected%) × ln(Actual% / Expected%)\n```\n\n**การแปลผล:**\n\n| PSI Value | Interpretation | Action |\n|-----------|----------------|--------|\n| < 0.10 | No significant change | Continue monitoring |\n| 0.10 - 0.25 | Slight change | Minor investigation |\n| ≥ 0.25 | Significant change | Major investigation/retrain |\n\n```python\ndef calculate_psi(self, reference_col, current_col, bins=10):\n    \"\"\"Population Stability Index\"\"\"\n    # สร้าง bins จาก reference\n    bin_edges = np.linspace(reference_col.min(), reference_col.max(), bins + 1)\n    \n    # คำนวณ % ในแต่ละ bin\n    ref_hist, _ = np.histogram(reference_col, bins=bin_edges)\n    curr_hist, _ = np.histogram(current_col, bins=bin_edges)\n    \n    ref_pct = ref_hist / len(reference_col)\n    curr_pct = curr_hist / len(current_col)\n    \n    # หลีกเลี่ยง log(0)\n    ref_pct = np.where(ref_pct == 0, 0.0001, ref_pct)\n    curr_pct = np.where(curr_pct == 0, 0.0001, curr_pct)\n    \n    # คำนวณ PSI\n    psi = np.sum((curr_pct - ref_pct) * np.log(curr_pct / ref_pct))\n    \n    return {\n        'psi': psi,\n        'interpretation': 'No Drift' if psi < 0.1 else \n                         'Slight Drift' if psi < 0.25 else \n                         'Significant Drift'\n    }\n```\n\n#### 3.2.4 Wasserstein Distance (Earth Mover's Distance)\n\n**Wasserstein Distance** วัดระยะห่างระหว่างสอง distributions โดยคิดจากปริมาณ \"work\" ที่ต้องใช้ในการเปลี่ยน distribution หนึ่งไปเป็นอีก distribution หนึ่ง\n\n```python\ndef wasserstein_distance_test(self, reference_col, current_col):\n    \"\"\"Wasserstein Distance\"\"\"\n    distance = wasserstein_distance(reference_col.dropna(), current_col.dropna())\n    \n    # Normalize by reference std\n    ref_std = reference_col.std()\n    normalized_distance = distance / ref_std if ref_std > 0 else distance\n    \n    return {\n        'distance': distance,\n        'normalized_distance': normalized_distance,\n        'drift_detected': normalized_distance > 0.1\n    }\n```\n\n### 3.3 Prediction Drift Detection\n\nนอกจากตรวจจับ drift ของ input data และ target แล้ว ยังต้องตรวจจับ **Prediction Drift** ด้วย:\n\n```python\ndef detect_prediction_drift(model, scaler, reference_data, current_data):\n    \"\"\"ตรวจจับ Prediction Drift\"\"\"\n    # ทำนาย probability\n    ref_proba = model.predict_proba(X_ref_scaled)[:, 1]\n    curr_proba = model.predict_proba(X_curr_scaled)[:, 1]\n    \n    # Statistical tests\n    ks_stat, ks_pval = ks_2samp(ref_proba, curr_proba)\n    wasserstein = wasserstein_distance(ref_proba, curr_proba)\n    \n    return {\n        'ks_statistic': ks_stat,\n        'ks_pvalue': ks_pval,\n        'wasserstein_distance': wasserstein,\n        'drift_detected': ks_pval < 0.05\n    }\n```\n\n### 3.4 Drift Detection Summary\n\n```\n┌──────────────────────────────────────────────────────────────────┐\n│                 DRIFT DETECTION DECISION MATRIX                  │\n├──────────────────────────────────────────────────────────────────┤\n│                                                                  │\n│  Variable Type    │  Recommended Test   │  Alternative           │\n│  ─────────────────┼─────────────────────┼────────────────────────│\n│  Continuous       │  KS Test + PSI      │  Wasserstein Distance  │\n│  Categorical      │  Chi-Square Test    │  Jensen-Shannon Div    │\n│  Binary Target    │  Chi-Square Test    │  Proportion Z-test     │\n│  Predictions      │  KS Test            │  Wasserstein Distance  │\n│                                                                  │\n└──────────────────────────────────────────────────────────────────┘\n```\n\n---\n\n## Section 4: Building Monitoring Dashboard\n\n### 4.1 ทฤษฎี: Dashboard Design Principles\n\n**Monitoring Dashboard** ที่ดีควรมีคุณสมบัติดังนี้:\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                   DASHBOARD DESIGN PRINCIPLES                   │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                 │\n│  1. VISIBILITY (เห็นภาพรวม)                                    │\n│     - Overall status at a glance                                │\n│     - Key metrics prominently displayed                         │\n│                                                                 │\n│  2. ACTIONABLE (นำไปปฏิบัติได้)                                │\n│     - Clear alerts with severity levels                         │\n│     - Recommended actions                                       │\n│                                                                 │\n│  3. HISTORICAL (มีประวัติ)                                     │\n│     - Trend visualization                                       │\n│     - Comparison with baseline                                  │\n│                                                                 │\n│  4. DRILL-DOWN (ดูรายละเอียดได้)                               │\n│     - From summary to detail                                    │\n│     - Root cause investigation                                  │\n│                                                                 │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n### 4.2 Dashboard Components\n\n```\n┌──────────────────────────────────────────────────────────────────┐\n│                    MODEL MONITORING DASHBOARD                    │\n├──────────┬────────────────────────────────────────┬──────────────┤\n│          │                                        │              │\n│  STATUS  │        PERFORMANCE METRICS             │   ALERTS     │\n│  🟢/🟡/🔴 │    ┌────────────────────────┐         │  ┌────────┐  │\n│          │    │  Accuracy  ████████ 92% │         │  │ 0 CRIT │  │\n│          │    │  Precision ███████  89% │         │  │ 2 WARN │  │\n│          │    │  Recall    ██████   85% │         │  └────────┘  │\n│          │    │  F1-Score  ███████  87% │         │              │\n│          │    └────────────────────────┘         │              │\n├──────────┴────────────────────────────────────────┴──────────────┤\n│                                                                  │\n│  DATA QUALITY                    │  DRIFT DETECTION              │\n│  ┌────────────────────┐          │  ┌────────────────────┐      │\n│  │ Missing:   ▓▓ 2%   │          │  │ Feature Drift: 3/10│      │\n│  │ Duplicate: ▓ 0.5%  │          │  │ Target Drift: NO   │      │\n│  │ Outliers:  ▓▓▓ 4%  │          │  │ Pred Drift: YES    │      │\n│  └────────────────────┘          │  └────────────────────┘      │\n│                                                                  │\n├──────────────────────────────────────────────────────────────────┤\n│                        TREND OVER TIME                           │\n│     100% ┤                                                       │\n│          │  ●──●──●──●──●                                       │\n│      80% ┤           ╲──●──●──●                                 │\n│          │                                                       │\n│      60% ┤                                                       │\n│          └──┬──┬──┬──┬──┬──┬──┬──►                              │\n│             W1 W2 W3 W4 W5 W6 W7 W8                              │\n└──────────────────────────────────────────────────────────────────┘\n```\n\n### 4.3 Comprehensive Dashboard Class\n\n```python\nclass ModelMonitoringDashboard:\n    \"\"\"Dashboard สำหรับ Model Monitoring\"\"\"\n    \n    def __init__(self, model, model_name=\"ML Model\"):\n        self.model = model\n        self.model_name = model_name\n        self.reports = {\n            'data_quality': {},\n            'performance': {},\n            'drift': {},\n            'alerts': []\n        }\n    \n    def run_full_monitoring(self, reference_data, current_data, target_col, scaler=None):\n        \"\"\"รัน monitoring ทั้งหมด\"\"\"\n        \n        # 1. Data Quality Monitoring\n        print(\"📊 [1/4] Running Data Quality Check...\")\n        dq_monitor = DataQualityMonitor(current_data)\n        self.reports['data_quality'] = {\n            'missing': dq_monitor.check_missing_values(),\n            'duplicates': dq_monitor.check_duplicates(),\n            'outliers': dq_monitor.detect_outliers()\n        }\n        \n        # 2. Performance Monitoring\n        print(\"📈 [2/4] Running Performance Evaluation...\")\n        # ... คำนวณ metrics ...\n        \n        # 3. Drift Detection\n        print(\"🔍 [3/4] Running Drift Detection...\")\n        drift_detector = DriftDetector(reference_data)\n        # ... ตรวจจับ drift ...\n        \n        # 4. Generate Alerts\n        print(\"🚨 [4/4] Generating Alerts...\")\n        self._generate_alerts()\n        \n        return self.reports\n```\n\n### 4.4 Alert Generation Logic\n\n```python\ndef _generate_alerts(self):\n    \"\"\"สร้าง alerts จากผลการ monitoring\"\"\"\n    self.reports['alerts'] = []\n    \n    # Data Quality Alerts\n    if missing_count > 0:\n        self.reports['alerts'].append({\n            'type': 'WARNING',\n            'category': 'Data Quality',\n            'message': 'พบ missing values ในข้อมูล'\n        })\n    \n    # Performance Alerts\n    degradation = (baseline_acc - current_acc) / baseline_acc * 100\n    if degradation > 10:\n        self.reports['alerts'].append({\n            'type': 'CRITICAL',\n            'category': 'Performance',\n            'message': f\"Accuracy ลดลง {degradation:.1f}%\"\n        })\n    \n    # Drift Alerts\n    if target_drift_detected:\n        self.reports['alerts'].append({\n            'type': 'CRITICAL',\n            'category': 'Drift',\n            'message': 'ตรวจพบ Target Drift'\n        })\n```\n\n### 4.5 Report Export\n\nการ export report เพื่อแชร์กับ stakeholders:\n\n```python\ndef export_report(self, filename=None):\n    \"\"\"Export report เป็น dictionary/JSON\"\"\"\n    return {\n        'model_name': self.model_name,\n        'timestamp': datetime.now().isoformat(),\n        'data_quality': {...},\n        'performance': {...},\n        'drift': {...},\n        'alerts': self.reports['alerts'],\n        'overall_status': 'CRITICAL' if any_critical else \n                         'WARNING' if any_warning else \n                         'HEALTHY'\n    }\n```\n\n---\n\n## Best Practices และแนวทางปฏิบัติ\n\n### 1. กำหนด Baseline ที่ชัดเจน\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                     BASELINE ESTABLISHMENT                      │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                 │\n│  ✓ บันทึก performance metrics เมื่อ deploy                     │\n│  ✓ เก็บ reference data distribution                            │\n│  ✓ กำหนด threshold สำหรับแต่ละ metric                          │\n│  ✓ Document assumptions และ limitations                        │\n│                                                                 │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n### 2. Monitor อย่างสม่ำเสมอ\n\n| Metric Type | Recommended Frequency |\n|-------------|----------------------|\n| Data Quality | Real-time / Daily |\n| Model Performance | Daily / Weekly |\n| Drift Detection | Weekly / Monthly |\n| Full Dashboard Review | Weekly |\n\n### 3. ตั้ง Threshold ที่เหมาะสม\n\n```python\n# Example threshold configuration\nthresholds = {\n    # Data Quality\n    'missing_rate': {'warning': 0.05, 'critical': 0.10},\n    'duplicate_rate': {'warning': 0.01, 'critical': 0.05},\n    \n    # Performance\n    'accuracy_degradation': {'warning': 0.05, 'critical': 0.10},\n    'f1_degradation': {'warning': 0.05, 'critical': 0.10},\n    \n    # Drift\n    'psi': {'warning': 0.10, 'critical': 0.25},\n    'ks_pvalue': {'threshold': 0.05}\n}\n```\n\n### 4. มี Action Plan\n\n```\n┌─────────────────────────────────────────────────────────────────┐\n│                     ALERT RESPONSE MATRIX                       │\n├─────────────────────────────────────────────────────────────────┤\n│                                                                 │\n│  Alert Level   │  Response Time  │  Action                      │\n│  ─────────────┼─────────────────┼──────────────────────────────│\n│  🟢 HEALTHY   │  -              │  Continue monitoring         │\n│  🟡 WARNING   │  < 24 hours     │  Investigate, prepare retrain│\n│  🔴 CRITICAL  │  < 4 hours      │  Immediate investigation     │\n│  ⚫ FATAL     │  Immediate      │  Rollback / Emergency retrain│\n│                                                                 │\n└─────────────────────────────────────────────────────────────────┘\n```\n\n### 5. Document Everything\n\n```python\n# Monitoring log structure\nmonitoring_log = {\n    'timestamp': '2024-01-15T10:30:00',\n    'model_version': 'v1.2.3',\n    'data_period': '2024-01-08 to 2024-01-14',\n    'metrics': {...},\n    'alerts': [...],\n    'actions_taken': 'None required',\n    'reviewed_by': 'ML Engineer'\n}\n```\n\n---\n\n## Quick Reference: Monitoring Checklist\n\n```\n□ Data Quality\n  □ Missing values < 5%\n  □ Duplicates < 1%\n  □ Outliers investigated\n  □ Schema validation passed\n\n□ Model Performance\n  □ Accuracy within acceptable range\n  □ Precision/Recall balanced\n  □ F1-Score stable\n  □ Confusion matrix reviewed\n\n□ Drift Detection\n  □ Feature distributions checked\n  □ Target drift assessed\n  □ Prediction drift analyzed\n  □ PSI < 0.10 for all features\n\n□ Alerts & Actions\n  □ All critical alerts addressed\n  □ Warning alerts investigated\n  □ Action plan documented\n  □ Stakeholders notified\n```\n\n---\n\n## สรุป\n\nModel Monitoring เป็นส่วนสำคัญของ MLOps ที่ช่วยให้มั่นใจว่าโมเดลทำงานได้อย่างถูกต้องหลัง deploy:\n\n1. **Data Quality Monitoring** - ตรวจสอบคุณภาพข้อมูลก่อนเข้าโมเดล\n2. **Performance Tracking** - ติดตามประสิทธิภาพโมเดลเทียบกับ baseline\n3. **Drift Detection** - ตรวจจับการเปลี่ยนแปลงของ data/target distribution\n4. **Dashboard & Alerts** - รวมทุกอย่างใน single view พร้อมระบบแจ้งเตือน\n\nการทำ monitoring ที่ดีช่วยให้ทีมสามารถตอบสนองต่อปัญหาได้รวดเร็ว และรักษา business value ของ ML models ไว้ได้อย่างต่อเนื่อง"
    }
  ],
 
}

In [6]:
# Simple extraction
text = response["content"][0]["text"]
print(text)

# Save to file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

# 📚 Model Monitoring with Scikit-Learn: Theory & Practice Guide

## สารบัญ (Table of Contents)
1. [บทนำ: Model Monitoring คืออะไร?](#1-บทนำ-model-monitoring-คืออะไร)
2. [Section 1: Data Quality Monitoring](#section-1-data-quality-monitoring)
3. [Section 2: Model Performance Tracking](#section-2-model-performance-tracking)
4. [Section 3: Target Drift Detection](#section-3-target-drift-detection)
5. [Section 4: Building Monitoring Dashboard](#section-4-building-monitoring-dashboard)
6. [Best Practices และแนวทางปฏิบัติ](#best-practices-และแนวทางปฏิบัติ)

---

## 1. บทนำ: Model Monitoring คืออะไร?

### 1.1 ความหมายและความสำคัญ

**Model Monitoring** คือกระบวนการติดตามและเฝ้าระวังประสิทธิภาพของโมเดล Machine Learning หลังจากที่ได้ Deploy ไปใช้งานจริง (Production) เพื่อให้มั่นใจว่าโมเดลยังคงทำงานได้อย่างถูกต้องและน่าเชื่อถือ

```
┌─────────────────────────────────────────────────────────────────┐
│                    ML Model Lifecycle                           │
├─────────────────────────────